### This data was obtain from instrument methods that were optimzed so there are no outliers.

In [520]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score,precision_recall_curve
import pandas as pd
import numpy as np


df = pd.read_csv("Linkage_Mono_data.csv")
feature_x1 = ['Glucose', 'Galactose', 'Fructose', 'Xylose', 'Arabinose',
       'Fucose', 'Rhamnose', 'GlcA', 'GalA', 'GlcNAc', 'GalNAc', 'Mannose',
       'Allose', 'Ribose', 'T-Glucose ', '4-Glucose ',
       '3-Glucose/3-Galactose ', '2-Glucose ', '6-Glucose ', '2,4-Glucose ',
       '3,4-Glucose ', '4,6-Glucose ', '2,4,6-Glucose ', '3,4,6-Glucose ',
       '2,3,6-Glucose ', 'T-Galactose ', '4-Galactose ', '2-Galactose ',
       '6-Galactose ', '3,6-Galactose ', '3,4-Galactose ', '4,6-Galactose ',
       '2,4,6-Galactose ', '3,4,6-Galactose ', 'T-Fructose ', 'T-P-Xylose ',
       '4-P-Xylose ', '2-Xylose ', '3,4-P-Xylose/3,5-Arabinose ',
       '2,4-P-Xylose ', 'T-P-Arabinose ', 'T-F-Arabinose ', '5-F-Arabinose ',
       '3-F-Arabinose ', '2-F-Arabinose ', '2,3-Arabinose ', '2,5-Arabinose ',
       'T-Fucose ', 'T-Rhamnose ', '2-Rhamnose ', '4-Rhamnose', 'T-GlcA ',
       'T-GalA ', 'T-Mannose ', '2-Mannose ', '4-Mannose ', '3-Mannose ',
       '6-Mannose ', '3,6-Mannose ', '4,6-Mannose ', '3,4,6-Mannose ',
       'X-Hexose ', '2,X,X-Hexose (I) ', '2,X,X-Hexose (II) ']
feature_x2 = ['T-Glucose ', '4-Glucose ',
       '3-Glucose/3-Galactose ', '2-Glucose ', '6-Glucose ', '2,4-Glucose ',
       '3,4-Glucose ', '4,6-Glucose ', '2,4,6-Glucose ', '3,4,6-Glucose ',
       '2,3,6-Glucose ', 'T-Galactose ', '4-Galactose ', '2-Galactose ',
       '6-Galactose ', '3,6-Galactose ', '3,4-Galactose ', '4,6-Galactose ',
       '2,4,6-Galactose ', '3,4,6-Galactose ', 'T-Fructose ', 'T-P-Xylose ',
       '4-P-Xylose ', '2-Xylose ', '3,4-P-Xylose/3,5-Arabinose ',
       '2,4-P-Xylose ', 'T-P-Arabinose ', 'T-F-Arabinose ', '5-F-Arabinose ',
       '3-F-Arabinose ', '2-F-Arabinose ', '2,3-Arabinose ', '2,5-Arabinose ',
       'T-Fucose ', 'T-Rhamnose ', '2-Rhamnose ', '4-Rhamnose', 'T-GlcA ',
       'T-GalA ', 'T-Mannose ', '2-Mannose ', '4-Mannose ', '3-Mannose ',
       '6-Mannose ', '3,6-Mannose ', '4,6-Mannose ', '3,4,6-Mannose ',
       'X-Hexose ', '2,X,X-Hexose (I) ', '2,X,X-Hexose (II) ']
feature_x3 = ['Glucose', 'Galactose', 'Fructose', 'Xylose', 'Arabinose',
       'Fucose', 'Rhamnose', 'GlcA', 'GalA', 'GlcNAc', 'GalNAc', 'Mannose',
       'Allose', 'Ribose']

### This was to normalize data based on chemical insight

In [521]:
count = 0
for i in df.columns:
    if i == 'Mono_starting mass (ng)':
        count += 1
        df['Linkage_starting mass (ng)'] = 4
    if count == 1:
        df[i] = df[i]/df['Linkage_starting mass (ng)'] *4
    if i == 'Ribose':
        count += 1

###  Used as an initial training to test logisitic regression but the precision and recall can vary greatly

In [540]:
lcassifier = LogisticRegression(multi_class= "multinomial", max_iter = 10000)
y = df["Sample"]
x = df[feature_x1]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)
lcassifier.fit(np.array(x_train),y_train)
pred = lcassifier.predict(np.array(x_test))
# print(metrics.classification_report(y_test, pred, digits = 3))
print("Precision:",precision_score(y_test,np.array(pred),average='macro',zero_division=0))
print("Recall:",recall_score(y_test,np.array(pred),average='macro',zero_division=0))

Precision: 0.7894736842105263
Recall: 0.7894736842105263


### Function used for logistic regression model using LOOCV. LOOCV is used because we only have 96 data points which wouldn't require large computational time. In addition, this also resolve the limitation of the data set since each data point only have 3 replicate experiments.

In [498]:
#function logistic_regression
def log_regress(x_train, y_train, x_test):
    lclassifier = LogisticRegression(multi_class= "multinomial", max_iter = 2000)
    lclassifier.fit(np.array(x_train),y_train)
    pred = lclassifier.predict(np.array(x_test).reshape(1,-1))
    return pred
def loocv_log(feature):
    lst = []
    for i, row in df.iterrows():
        x_train = df[feature].drop([i])
        y_train = df["Sample"].drop([i])
        x_test = row[feature]
        lst.append(log_regress(x_train,y_train,x_test)[0])
    return lst

### LOOCV to test the 3 feature. This was done to see if feasture set 2 would be sufficiant prediction. This means we would reduce our instrumental analysis needed for future development of the model.

In [532]:

lst_1x = loocv_log(feature_x1)
# print(metrics.classification_report(df["Sample"], np.array(lst_1x), digits = 3))
print("Precision:",precision_score(df["Sample"],np.array(lst_1x),average='macro',zero_division=0))
print("Recall:",recall_score(df["Sample"],np.array(lst_1x),average='macro',zero_division=0))

Precision: 0.9493087557603688
Recall: 0.913978494623656


In [531]:
#loocv for feature set 2
lst_2x = loocv_log(feature_x2)
# print(metrics.classification_report(df["Sample"], np.array(lst_2x), digits = 3))
print("Precision:",precision_score(df["Sample"],np.array(lst_2x),average='macro',zero_division=0))
print("Recall:",recall_score(df["Sample"],np.array(lst_2x),average='macro',zero_division=0))

Precision: 0.957373271889401
Recall: 0.924731182795699


In [530]:
#loocv for feature set 3
lst_3x = loocv_log(feature_x3)
# print(metrics.classification_report(df["Sample"], np.array(lst_3x), digits = 3))
print("Precision:",precision_score(df["Sample"],np.array(lst_3x),average='macro',zero_division=0))
print("Recall:",recall_score(df["Sample"],np.array(lst_3x),average='macro',zero_division=0))

Precision: 0.003795066413662239
Recall: 0.03225806451612903


### Repeated the same procress for linear regression.

In [543]:
#initial training for linear regression using 80/20 split
cassifier = LinearRegression()
encode = OneHotEncoder()
encode.fit(np.array(df['Sample']).reshape(-1,1))
encode.categories
df2 = pd.DataFrame(encode.transform(np.array(df['Sample']).reshape(-1,1)).toarray())
final_df = pd.concat([df2,df[feature_x2]], axis = 1)
test_df = pd.concat([df2,df3[feature_x2]], axis =1)
y = df['Mono_starting mass (ng)']
x = final_df
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)
cassifier.fit(np.array(x_train),y_train)
pred = cassifier.predict(np.array(x_test))
# print(pred)
# print(metrics.classification_report(y_test, pred, digits = 3))
# print("Precision:",precision_score(y_test,np.array(pred),average='macro',zero_division=0))
# print("Recall:",recall_score(y_test,np.array(pred),average='macro',zero_division=0))

In [499]:
def lin_regress(x_train, y_train, x_test):
    cassifier = LinearRegression()
    cassifier.fit(np.array(x_train),y_train)
    pred = cassifier.predict(np.array(x_test).reshape(1,-1))
    return pred
def loocv_lin(feature):
    lst = []
    encode = OneHotEncoder()
    encode.fit(np.array(df['Sample']).reshape(-1,1))
    df_encode = pd.DataFrame(encode.transform(np.array(df['Sample']).reshape(-1,1)).toarray())
    final_df = pd.concat([df_encode,df[feature]], axis = 1)
    for i, row in final_df.iterrows():
        x_train = final_df.drop([i])
        y_train = df["Mono_starting mass (ng)"].drop([i])
        x_test = row
        lst.append(lin_regress(x_train,y_train,x_test)[0])
    return lst

In [527]:
lst_4x = loocv_lin(feature_x1)
# print(metrics.classification_report(df["Mono_starting mass (ng)"], np.array(lst_4x), digits = 3))
# print("Precision:",precision_score(df["Mono_starting mass (ng)"],np.array(lst_4x),average='macro',zero_division=0))
# print("Recall:",recall_score(df["Mono_starting mass (ng)"],np.array(lst_4x),average='macro',zero_division=0))

In [528]:
lst_5x = loocv_lin(feature_x2)
# print(metrics.classification_report(df["Mono_starting mass (ng)"], np.array(lst_5x), digits = 3))
# print("Precision:",precision_score(df["Mono_starting mass (ng)"],np.array(lst_5x),average='macro',zero_division=0))
# print("Recall:",recall_score(df["Mono_starting mass (ng)"],np.array(lst_5x),average='macro',zero_division=0))

In [529]:
lst_6x = loocv_lin(feature_x3)
# print(metrics.classification_report(df["Mono_starting mass (ng)"], np.array(lst_6x), digits = 3))
# print("Precision:",precision_score(df["Mono_starting mass (ng)"],np.array(lst_6x),average='macro',zero_division=0))
# print("Recall:",recall_score(df["Mono_starting mass (ng)"],np.array(lst_6x),average='macro',zero_division=0))